## This is the Maxwell part of the system, how to compute the density and then the potential, to finally compute the electric field.

In [ ]:
using Plots
using FFTW
FFTW.set_provider!("mkl")
using LinearAlgebra


In [ ]:
include("aux_functions.jl")

We try a density and compute the potential. Then check it is working

In [ ]:
J = 128
L = 3.
dx = L/J
κ = 2π/L
x = [dx*(i-1) for i in 1:J]
ω = 5
v = sin.(κ*ω*x) + 3. * cos.(κ*4*ω*x)
ϕe = -(κ *ω )^(-2)*(sin.(κ*ω*x) + 3.0/4/4 * cos.(κ*4*ω*x))
u = zeros(J)
ϕ = get_ϕ!(u,v,κ);
ϕ_D = get_ϕ_D!(u,v,(0.0,L))

In [ ]:
plot(x,ϕ_D)
plot!(x,ϕe ,ls=:dash)

In [ ]:
plot(x, ϕ_D - ϕe)

Finally we check that the Electric field is computed correctly.

In [ ]:
E = zeros(J)
Electric!(ϕ, E, dx)
plot(x,E)

### 2D check

In [ ]:
include("aux_functions.jl")

J = (200,100)
Box = (0.0,1.0,0.0,2.0)
x = Box[1]:0.01:Box[2]
y = Box[3]:0.01:Box[4]

ρ = Array{Float64,2}(undef,J)

Box_array = [i for i in Box]
κ = 2π./(Box_array[2:2:end] - Box_array[1:2:end-1])

dx = (Box_array[2:2:end] - Box_array[1:2:end-1])./(J)

x = [Box[1] + (i-1)*dx[1]*(Box[2]-Box[1]) for i in 1:J[1]]
y = [Box[3] + (i-1)*dx[2]*(Box[4]-Box[3]) for i in 1:J[2]]

@show κ
@show k = [2,2].*κ

@show k'*k

#ρ = sin.(k[1]*x) * sin.(k[2]*y)'

for i in 1:J[1]
    for j in 1:J[2]
        ρ[i,j] = sin(k[1]*(i-1)*dx[1])*sin(k[2]*(j-1)*dx[2])
        #ρ[i,j] = sin(k[1]*(i-1)*dx[1])
        #ρ[i,j] = sin(k[2]*(j-1)*dx[2])
    end
end

ϕe = similar(ρ)

for i in 1:J[1]
    for j in 1:J[2]
        ρ[i,j] = sin(k[1]*(i-1)*dx[1])*sin(k[2]*(j-1)*dx[2])
        #ρ[i,j] = sin(k[1]*(i-1)*dx[1])
        #ϕe[i,j] = -k[2]^(-2)*sin(k[2]*(j-1)*dx[2])
        ϕe[i,j] = -(k[1]^2 + k[2]^2)^(-1)*sin(k[1]*(i-1)*dx[1])*sin(k[2]*(j-1)*dx[2])
    end
end

ϕ = similar(ρ)

#plot_matrix(ρ)

surface(y[1:end],x[1:end],ρ)

#plot(x,ρ[:,25])
#length(x)

In [ ]:
include("aux_functions.jl")
get_ϕ_D!(ϕ, ρ, Box)

#plot_matrix(ϕ)

surface(y,x,ϕ-ϕe)
#surface!(y,x,ϕe)
#plot(y,ϕ[45,:])
#plot!(y,ϕe[45,:], ls=:dash)

In [ ]:
maximum(ϕ)*(k'*k)

In [ ]:
norm(ϕ - ϕe)

### D = 3 Check

In [ ]:
include("aux_functions.jl")

J = (200,100,300)
Box = (0.0,1.0,0.0,2.0,1.0,4.0)

ρ = Array{Float64,3}(undef,J)

Box_array = [i for i in Box]
κ = 2π./(Box_array[2:2:end] - Box_array[1:2:end-1])

@show dx = (Box_array[2:2:end] - Box_array[1:2:end-1])./(J)

x = [Box[1] + (i-1)*dx[1]*(Box[2]-Box[1]) for i in 1:J[1]]
y = [Box[3] + (i-1)*dx[2]*(Box[4]-Box[3]) for i in 1:J[2]]
z = [Box[5] + (i-1)*dx[3]*(Box[6]-Box[5]) for i in 1:J[3]]

@show κ
@show kv = [2,2,4].*κ

@show kv'*kv

#ρ = sin.(k[1]*x) * sin.(k[2]*y)'

for i in 1:J[1]
    for j in 1:J[2]
        for k in 1:J[3]
            ρ[i,j,k] = sin(kv[1]*(i-1)*dx[1])*sin(kv[2]*(j-1)*dx[2])*sin(kv[3]*(k-1)*dx[3])
        end
        #ρ[i,j] = sin(k[1]*(i-1)*dx[1])
        #ρ[i,j] = sin(k[2]*(j-1)*dx[2])
    end
end

ϕe = similar(ρ)

for i in 1:J[1]
    for j in 1:J[2]
        for k in 1:J[3]
            ϕe[i,j,k] = -(kv[1]^2 + kv[2]^2 + kv[3]^2)^(-1)*sin(kv[1]*(i-1)*dx[1])*sin(kv[2]*(j-1)*dx[2])*sin(kv[3]*(k-1)*dx[3])
        end
    end
end



#plot_matrix(ρ)

surface(y,x,ρ[:,:,50])

#plot(x,ρ[:,25,30])
#length(x)

In [ ]:
ϕ = similar(ρ);
get_ϕ_D!(ϕ, ρ, Box);

In [ ]:
norm(ϕ - ϕe)

In [ ]:
surface(y,x,(ϕ-ϕe)[:,:,50])

## Checking getting E in one go

### 1D

In [ ]:
include("aux_functions.jl")
J = 128
L = 3.
dx = L/J
κ = 2π/L
x = [dx*(i-1) for i in 1:J]
ω = 5
ρ = sin.(κ*ω*x) + 3. * cos.(κ*4*ω*x)
Ee = (κ *ω )^(-2)*(κ*ω*cos.(κ*ω*x) - 3.0/4/4*κ*ω*4 * sin.(κ*4*ω*x))
E = zeros(J)
E = get_E!(E,ρ,(0.0,L))

In [ ]:
#plot(E-Ee)
plot(E)

### D = 2

In [ ]:
include("aux_functions.jl")

J = (200,100)
Box = (0.0,1.0,0.0,2.0)
x = Box[1]:0.01:Box[2]
y = Box[3]:0.01:Box[4]

ρ = Array{Float64,2}(undef,J)

Box_array = [i for i in Box]
κ = 2π./(Box_array[2:2:end] - Box_array[1:2:end-1])

dx = (Box_array[2:2:end] - Box_array[1:2:end-1])./(J)

x = [Box[1] + (i-1)*dx[1]*(Box[2]-Box[1]) for i in 1:J[1]]
y = [Box[3] + (i-1)*dx[2]*(Box[4]-Box[3]) for i in 1:J[2]]

@show κ
@show k = [2,2].*κ

@show k'*k

#ρ = sin.(k[1]*x) * sin.(k[2]*y)'

for i in 1:J[1]
    for j in 1:J[2]
        ρ[i,j] = sin(k[1]*(i-1)*dx[1])*sin(k[2]*(j-1)*dx[2])
        #ρ[i,j] = sin(k[1]*(i-1)*dx[1])
        #ρ[i,j] = sin(k[2]*(j-1)*dx[2])
    end
end

Ee = Array{Float64,3}(undef,(2,J...))

for i in 1:J[1]
    for j in 1:J[2]
        Ee[1,i,j] = k[1]*(k[1]^2 + k[2]^2)^(-1)*cos(k[1]*(i-1)*dx[1])*sin(k[2]*(j-1)*dx[2])
        Ee[2,i,j] = k[2]*(k[1]^2 + k[2]^2)^(-1)*sin(k[1]*(i-1)*dx[1])*cos(k[2]*(j-1)*dx[2])
    end
end

E = Array{Float64,3}(undef,(2,J...))

#plot_matrix(ρ)

#surface(y[1:end],x[1:end],ρ)

#plot(x,ρ[:,25])
#length(x)

In [ ]:
get_E!(E,ρ,Box)

In [ ]:
norm(E-Ee)

In [ ]:
surface(E[1,:,:])